# Proposal to add sloped terrain not parallel to tracker axes

The following snippets demonstrate how single axis trackers can be modified to accomodate sopes not parallel to the tracker axis.

## System Plane
The system plane is a new parameter that is a tuple representing the azimuth and zenith of the normal to the plane that contains the tracker axes. It is called the system plane and not the slope or terrain plane, because it is defined by the plane that contains the tracker axes, even though the trackers might not be parallel to the system azimuth. EG: An E-W slope with N-S trackers; the slope normal faces either east or west, while the tracker axes are N-S.

## Calculated Axis Tilt, Side Slope, and Relative Rotation
Three new functions are added to calculated to adapt the single axis tracker to accomodate the system plane:
* the **tracker axis tilt** is calculated as a function of the tracker azimuth and the system plane
* the **side slope** is the slope of the system perpendicular to the tracker axes
* the **relative rotation** is the azimuth of the tracker axes relative to the system plane azimuth

## Proposed Changes
Once the tracker axis tilt, $\theta_{tracker}$, is calculated then the existing `pvlib.tracking.singleaxis()` function can be used by setting the tracker axis to the calculated value. However two additional changes must be made to account for the system plane if it is not parallel to the tracker axes:

* the condition for backtracking and the rotation calculation should have the side slope applied

* the max angle should also be applied with the side slope in consideration

With these 2 changes in `pvlib.tracking.singleaxis()`, the proposed 2 new functions for calculating **axis tilt**, **side slope**, and **relative rotation**, and the concept of the **system plane**, then pvlib-python can handle trackers on sloped terrain, which is an emerging topic encountered in many PV projects in 2019-2020.

### Axis Tilt
Solving for the tracker tilt, $\theta_{tracker}$, on a slope is derived in the following steps:

1. the slope is the system plane normal, defined by an azimuth, zenith pair: $\left(\phi_{sys}, \theta_{sys}\right)$

2. the trackers axes are in the system plane, so $z_{tracker,sys} = 0$

3. rotate the trackers $\left[x_{tracker,sys}, y_{tracker,sys}, 0\right]$ **back** from the system to the global reference frame, but rotated by the tracker global azimuth, $\phi_{tracker}$, if not zero, so that the tracker axis is constrained to y-z plane of the global reference frame and $x_{tracker,glo} = 0$

$$R_{x,sys} = \left[ { \begin{array}{ccc}
1&           0&            0\\
0& \cos\left(\theta_{sys}\right)& -\sin\left(\theta_{sys}\right)\\
0& \sin\left(\theta_{sys}\right)&  \cos\left(\theta_{sys}\right)
\end{array} }\right]$$

$$R_{z,sys} = \left[ { \begin{array}{ccc}
\cos\left(\phi_{sys} - \phi_{tracker}\right)& -\sin\left(\phi_{sys} - \phi_{tracker}\right)& 0\\
\sin\left(\phi_{sys} - \phi_{tracker}\right)&  \cos\left(\phi_{sys} - \phi_{tracker}\right)& 0\\
                0&                  0& 1
\end{array} }\right]$$

$$tracker_{glo} = R_{z,sys}^{T} \left(R_{x,sys}^{T} \left[x_{tracker,sys}, y_{tracker,sys}, 0\right]\right)$$

$$tracker_{glo} = \left[ { \begin{array}{ccc}
 x_{tracker,sys}\cos\left(\phi_{sys} - \phi_{tracker}\right) + y_{tracker,sys}\sin\left(\phi_{sys} - \phi_{tracker}\right)\cos\left(\theta_{sys}\right)\\
-x_{tracker,sys}\sin\left(\phi_{sys} - \phi_{tracker}\right) + y_{tracker,sys}\cos\left(\phi_{sys} - \phi_{tracker}\right)\cos\left(\theta_{sys}\right)\\
-y_{tracker,sys}\sin\left(\theta_{sys}\right)
\end{array} }\right]$$


4. solve for $x_{tracker,sys}$

$$x_{tracker,sys}\cos\left(\phi_{sys} - \phi_{tracker}\right) + y_{tracker,sys}\sin\left(\phi_{sys} - \phi_{tracker}\right)\cos\left(\theta_{sys}\right) = 0$$
$$x_{tracker,sys} = -y_{tracker,sys}\tan\left(\phi_{sys} - \phi_{tracker}\right)\cos\left(\theta_{sys}\right)$$

5. the tracker axis tilt is $\theta_{tracker} = \tan^{-1}\left(\frac{z_{tracker,glo}}{ y_{tracker,glo}}\right)$, so substituting in the solution for $x_{tracker,glo}$ from the previous step into the expression for $tracker_{glo}$, we can solve for $y_{tracker,glo}$:

$$y_{tracker,glo} = y_{tracker,sys} \cos \left( \theta_{sys} \right) \left(\tan\left(\phi_{sys} - \phi_{tracker}\right)\sin\left(\phi_{sys} - \phi_{tracker}\right) + \cos\left(\phi_{sys} - \phi_{tracker}\right)\right)$$

6. then substituting for $z_{tracker,glo}$ from the expression for $tracker_{glo}$, we can solve for $\tan\left(\theta_{tracker}\right)$:

$$\tan\left(\theta_{tracker}\right) = \frac{-y_{tracker,sys}\sin\left(\theta_{sys}\right)}{y_{tracker,glo}}$$

The trick is multiply top and bottom by $\cos\left(\phi_{sys} - \phi{tracker}\right)$ and remember that $\sin^2 + \cos^2 = 1$, then finally:

$$\tan\left(\theta_{tracker}\right) = -\tan\left(\theta_{sys}\right)\cos\left(\phi_{sys} - \phi_{tracker}\right) $$

Now the tracker axis tilt can be used in the existing `pvlib.tracking.singleaxis()` function as long as the side slope is considered in the max angle and backtracking calculations.

### Side Slope
There's more than one way to calculate the side slope, but this derivation will use rotations. In order to calculate the tracker side slope with rotations, we will need the relative rotation of the tracker axes in the system plane, so we'll calculate that first.

#### Relative Rotation
1. first create a vector representing the tracker axis in the global reference frame, but this time, do not remove the tracker azimuth as we did in deriving the axis tilt

$$tracker_{glo} = \left[ { \begin{array}{c}
\cos\left(-\theta_{tracker}\right)\sin\left(\phi_{tracker}\right)\\
\cos\left(-\theta_{tracker}\right)\cos\left(\phi_{tracker}\right)\\
\sin\left(-\theta_{tracker}\right)
\end{array} }\right]$$

2. now rotate the tracker axes into the system plane using rotation matrices, but don't rotate around the global z-axis by $\phi_{tracker}$ like we did in the deriving the axis tilt

$$R_{z,sys} = \left[ { \begin{array}{ccc}
\cos\left(\phi_{sys} \right)& -\sin\left(\phi_{sys} \right)& 0\\
\sin\left(\phi_{sys} \right)&  \cos\left(\phi_{sys} \right)& 0\\
                0&                  0& 1
\end{array} }\right]$$

$$R_{x,sys} = \left[ { \begin{array}{ccc}
1&           0&            0\\
0& \cos\left(\theta_{sys}\right)& -\sin\left(\theta_{sys}\right)\\
0& \sin\left(\theta_{sys}\right)&  \cos\left(\theta_{sys}\right)
\end{array} }\right]$$

$$tracker_{sys} = R_{z,sys} \cdot \left( R_{x,sys} \cdot tracker_{glo} \right)$$

3. now that tracker axes are in the system plane the relative rotation is the arctangent of the x and y components, although we'll use [`atan2(x, y)`](https://en.wikipedia.org/wiki/Atan2) to make sure that the angle is correct in all four quadrants

$$\phi_{tracker,relative} = tan_2^{-1}\left(x_{tracker,sys}, y_{tracker,sys}\right) = tan^{-1}\left(\frac{x_{tracker,sys}}{ y_{tracker,sys}}\right)$$

#### Side Slope Calculation
Now that we have the relative rotation, we can calculate the side slope by rotating the tracker axis normal into the system plane. The side slope is the angle of the tracker normal from the system vertical axis.

1. create a vector representing the tracker axis normal in the global reference frame, it should be perpendicular to the tracker axis

$$tracker_{\perp,glo} = \left[ { \begin{array}{c}
\sin\left(\theta_{tracker}\right)\sin\left(\phi_{tracker}\right)\\
\sin\left(\theta_{tracker}\right)\cos\left(\phi_{tracker}\right)\\
\cos\left(\theta_{tracker}\right)
\end{array} }\right]$$

2. now rotate the tracker normal to the system plane using the system rotation matrices just derived:

$$tracker_{\perp,sys} = R_{z,sys} \cdot \left( R_{x,sys} \cdot tracker_{\perp,glo} \right)$$

3. the side slope is just the angle between the z-component of the tracker normal (a unit vector) and the system vertical axis, or $cos^{-1}\left(z_{tracker\perp}\right)$, but we also need to know which _direction_ the side slope is so first we rotate the normal by the relative rotation to remove the y-component which would be aligned with the tracker axes

$$R_{z,tracker,relative} = \left[ { \begin{array}{ccc}
\cos\left(\phi_{tracker,relative} \right)& -\sin\left(\phi_{tracker,relative} \right)& 0\\
\sin\left(\phi_{tracker,relative} \right)&  \cos\left(\phi_{tracker,relative} \right)& 0\\
                0&                  0& 1
\end{array} }\right]$$

$$tracker_{\perp,relative} = R_{z,tracker,relative} \cdot tracker_{\perp,sys}$$
$$\theta_{\perp,tracker,sys} = \tan^{-1}\left(\frac{x_{\perp,tracker,relative}}{z_{\perp,tracker,relative}}\right)$$

Now apply the tracker side slope relative to the system plane to the max angle and backtracking.

### Backtracking
When north-south trackers are on a slope that has an east-west component, backtracking will not be symmetrical. _IE_: if the slope faces east and the trackers run north to south, then backtracking will occur for a much shorter time in the morning than in the evening. If we apply the side slope, $\theta_{\perp,tracker,sys}$, to the tracker rotation, $R$, then the trackers are in the system reference frame, and we can determine backtracking as if the trackers were on the horizontal.

![backtracking](./backtracking.png)

From the figure, the trackers will shade each other if the length of the shadow cast on the ground, $L_x$ is greater than the distance between the trackers, $x$. This condition, $L_x > x$, is equivalent to when $\cos\left(R\right) < \text{GCR}$ because $\text{GCR} = \frac{L}{x}$ and $\cos\left(R\right) = \frac{L}{L_x}$ where $R$ is the tracker rotation which optimizes the aperture of the PV surface normal to the sun rays, $L$ is the length of the tracker perpendicular to its axes, and GCR is the ground coverage ratio. Just add the side slope to the rotation to take the slope of the system plane into account.

$$L_R = \cos\left(R+\theta_{\perp,tracker,sys}\right)$$
$$\text{backtracking} = L_R < \text{GCR}$$

Then the optimum backtracking angle correction, $R_{backtrack}$, denoted as $R - \theta$ in the figure above, is determined by recalling that a triangle inscripted in a semicircle is a right triangle, and therefore $\cos\left(R_{backtrack}\right) = \frac{x\cos\left(R\right)}{L}$ where $L$ is the length of the PV surface, but $\text{GCR} = \frac{L}{x}$, so the backtracking correction can be evaluated with the following expression.

$$R_{backtrack} = \cos^{-1}\left(\frac{L_R}{\text{GCR}}\right)$$

The arccosine is always positive between -90&deg; and 90&deg;, so apply it correctly, call the `sign()` function on the tracker rotation and apply backtracking in the opposite direction.

$$R_{backtracking} = R - R_{backtrack} \text{sign}\left(R\right)$$

### Max Angle
The max rotation angles, $\theta_{max}$ are also affected by the side slope, $\theta_{\perp,tracker,sys}$. If we apply the side slope, $\theta_{\perp,tracker,sys}$, to the tracker rotation, $R$, then the trackers are in the system reference frame, and we can determine backtracking as if the trackers were on the horizontal.

$$R_{max} = \min\left(\max\left(-\theta_{max}, R+\theta_{\perp,tracker,sys}\right), \theta_{max}\right)$$

# Examples
We can test if this is working by comparing it to SolarFarmer, assuming SF is correct.

In [1]:
import numpy as np
import pandas as pd
import pvlib
from singleaxis_tracking_wslope_helpers import calc_tracker_axis_tilt, calc_system_tracker_side_slope

system_plane = (77.34, 10.1149)
system_azimuth = np.radians(system_plane[0])
system_zenith = np.radians(system_plane[1])
axis_azimuth = 0
tracker_azimuth = np.radians(axis_azimuth)
max_rotation = 75.0
gcr = 0.328
tracker_zenith = calc_tracker_axis_tilt(system_azimuth, system_zenith, tracker_azimuth)
axis_tilt = np.degrees(tracker_zenith)
axis_tilt

2.2390131691716797

In [2]:
side_slope, tr_rel_rot = calc_system_tracker_side_slope(
    tracker_azimuth, tracker_zenith, system_azimuth, system_zenith)
print(f'side slope = {np.degrees(side_slope)}, relative rotation = {np.degrees(tr_rel_rot)}')

side slope = -9.866492718894921, relative rotation = -77.14671562495475
